In [1]:
import pandas as pd
from gensim import corpora, models
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Load the data
data = pd.read_csv('your_file.csv')

# Preprocess the text
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

data['Processed_Text'] = data['Questions'].apply(preprocess_text)

# Create a dictionary and a corpus
dictionary = corpora.Dictionary(data['Processed_Text'])
corpus = [dictionary.doc2bow(text) for text in data['Processed_Text']]

# Train the LSI model
lsi_model = models.LsiModel(corpus, id2word=dictionary, num_topics=4)

# Extract topics
topics = lsi_model.show_topics(num_topics=4, num_words=10)

for topic_id, topic in topics:
    print(f"Topic {topic_id}: {topic}")

# Assign questions to clusters
def get_topic(question):
    bow = dictionary.doc2bow(preprocess_text(question))
    topic_distribution = lsi_model[bow]
    dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]
    return dominant_topic

data['Cluster'] = data['Questions'].apply(get_topic)

# Print the first few rows with the assigned cluster
print(data[['Questions', 'Cluster']].head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RVCCF011\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv("spam.csv",encoding="Windows-1252")
X = df['v2']
y = df['v1'].apply(lambda x: 1 if x == "ham" else 0)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

stop_words = set(stopwords.words('english'))
preprocess = lambda text: ' '.join([word.lower() for word in word_tokenize(text) if word.isalpha() and word.lower() not in stop_words])
X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)

def calculate_tf(term, document):
    words = document.split()
    return words.count(term) / (len(words)+1)

def calculate_idf(term, documents):
    document_containing_term = sum(1 for document in documents if term in document.split())
    return math.log(len(documents) / (document_containing_term + 1)) if document_containing_term > 0 else 0

all_documents = X_train.tolist() + X_test.tolist()
idf_values = {term: calculate_idf(term, all_documents) for term in set(' '.join(all_documents).split())}

vocabulary = sorted(list(idf_values.keys()))

X_train_tfidf_manual = []
for document in X_train:
    tfidf_vector = [calculate_tf(term, document) * idf_values[term] for term in vocabulary]
    X_train_tfidf_manual.append(tfidf_vector)

X_test_tfidf_manual = []
for document in X_test:
    tfidf_vector = [calculate_tf(term, document) * idf_values[term] for term in vocabulary]
    X_test_tfidf_manual.append(tfidf_vector)

X_train_tfidf_manual = np.array(X_train_tfidf_manual)
X_test_tfidf_manual = np.array(X_test_tfidf_manual)

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
model = RandomForestClassifier()
model.fit(X_train_tfidf_manual, y_train)
y_pred = model.predict(X_test_tfidf_manual)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.80      0.89       219
           1       0.97      1.00      0.99      1453

    accuracy                           0.97      1672
   macro avg       0.99      0.90      0.94      1672
weighted avg       0.98      0.97      0.97      1672



In [5]:
new_text = "I HAVE A DATE ON SUNDAY WITH WILL!!"
new_text_tfidf_manual = [calculate_tf(term, preprocess(new_text)) * idf_values[term] for term in vocabulary]
predicted_sentiment = model.predict([new_text_tfidf_manual])
print("Predicted Sentiment:", "Not Spam" if predicted_sentiment[0] == 1 else "Spam")

Predicted Sentiment: Not Spam


In [6]:
new_text = "URGENT! You have won a 1 week FREE membership in our �100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18"

new_text_tfidf_manual = [calculate_tf(term, preprocess(new_text)) * idf_values[term] for term in vocabulary]
predicted_sentiment = model.predict([new_text_tfidf_manual])
print("Predicted Sentiment:", "Not Spam" if predicted_sentiment[0] == 1 else "Spam")

Predicted Sentiment: Spam
